# Delta Lake

## Time Travel

In [0]:
-- UC のパーソナル領域
USE CATALOG users;
USE SCHEMA yukiteru_koide;


In [0]:
-- デモ用テーブル（CDFは今回は不要）
CREATE OR REPLACE TABLE deltalake (
  id       INT,
  amount   DECIMAL(10,2),
  country  STRING,
  ts       TIMESTAMP
) USING DELTA;

INSERT INTO deltalake VALUES
  (1, 100.00, 'JP', current_timestamp()),
  (2, 200.00, 'US', current_timestamp()),
  (3, 150.00, 'JP', current_timestamp());

SELECT * FROM deltalake ORDER BY id;


In [0]:
-- （わざと）誤操作：JPを消してしまう
DELETE FROM deltalake WHERE country = 'JP';




In [0]:
-- 履歴の確認：オペレーションとバージョンを把握
DESCRIBE HISTORY deltalake;



In [0]:
-- 過去の状態を参照
SELECT * FROM deltalake VERSION AS OF 3 ORDER BY id;



In [0]:
-- 直近の良い状態へ一発復旧（RESTORE）
-- ※ 履歴を見て実際のバージョン番号に合わせてください
RESTORE TABLE deltalake TO VERSION AS OF 3;



In [0]:
SELECT * FROM deltalake ORDER BY id;  -- 復旧確認

## VACUUM（ストレージ容量の確保）

In [0]:
-- 履歴を確認
DESCRIBE HISTORY users.yukiteru_koide.deltalake;

In [0]:
-- 過去バージョンを参照できる（VACUUM前）
SELECT * FROM users.yukiteru_koide.deltalake VERSION AS OF 3;

In [0]:
-- DRY RUNで削除予定ファイルを確認（実際には消さない）
VACUUM users.yukiteru_koide.deltalake DRY RUN;

In [0]:
ALTER TABLE users.yukiteru_koide.deltalake
  SET TBLPROPERTIES ('delta.deletedFileRetentionDuration' = '1 hours');

In [0]:

VACUUM users.yukiteru_koide.deltalake;

In [0]:
-- VACUUM後の状態確認
DESCRIBE HISTORY users.yukiteru_koide.deltalake;

In [0]:
-- ここで古いバージョン参照を試す（ファイルが物理削除済みならエラーになる想定）
-- ※ 0以外の古いバージョン番号があればそれを使用
SELECT * FROM users.yukiteru_koide.deltalake VERSION AS OF 0;

In [0]:
-- 保持期間設定を元に戻す（推奨）
ALTER TABLE users.yukiteru_koide.deltalake
  UNSET TBLPROPERTIES ('delta.deletedFileRetentionDuration');

## Liquid Clustering（パフォーマンス・チューニング）

In [0]:
-- 現状の確認
DESCRIBE EXTENDED users.yukiteru_koide.deltalake;

In [0]:
-- CLUSTER BY の指定
ALTER TABLE users.yukiteru_koide.deltalake
CLUSTER BY (country);

In [0]:
-- 再度メタデータ確認
DESCRIBE EXTENDED users.yukiteru_koide.deltalake;